In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
from PIL import Image
import os

class SignClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SignClassifier, self).__init__()
        # MobileNetV3 모델을 가져와 수정
        self.model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT)
        # Adaptive Average Pooling 추가하여 고정된 크기의 출력 얻기
        self.model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # 기존의 classifier를 제거하고 새로운 다층 구조 추가
        self.model.classifier = nn.Sequential(
            nn.Linear(576, 128),  # Adaptive Pooling 후 크기 576을 사용
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout 추가
            nn.Linear(128, 64),  # 두 번째 레이어
            nn.ReLU(),
            nn.Linear(64, num_classes)  # 마지막 레이어
        )

    def forward(self, x):
        x = self.model.features(x)  # 특징 추출 부분
        x = self.model.avgpool(x)   # Adaptive Pooling으로 크기 고정
        x = torch.flatten(x, 1)  # (batch_size, 576)으로 평탄화
        x = self.model.classifier(x)  # 새로운 classifier 통과
        return x



# 클래스 수 설정 (우회전, 천천히, 진입금지, 좌회전금지)
num_classes = 4
model = SignClassifier(num_classes)

# 데이터셋 경로 설정
train_data_dir = '/content/drive/MyDrive/2024 덕성여대 졸업 프로젝트/표지판 데이터셋/CNN 데이터셋/trainCNN_Nomal'
valid_data_dir = '/content/drive/MyDrive/2024 덕성여대 졸업 프로젝트/표지판 데이터셋/CNN 데이터셋/validCNN_Nomal'
test_data_dir = '/content/drive/MyDrive/2024 덕성여대 졸업 프로젝트/표지판 데이터셋/CNN 데이터셋/testCNN_Nomal'

# 데이터 전처리 및 로더 설정
# Train 데이터에 대해 리사이즈와 정규화 적용
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 리사이즈 추가
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Valid와 Test 데이터에 대해 리사이즈와 정규화 적용
valid_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 리사이즈
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.ImageFolder(train_data_dir, transform=train_transform)
valid_dataset = datasets.ImageFolder(valid_data_dir, transform=valid_test_transform)
test_dataset = datasets.ImageFolder(test_data_dir, transform=valid_test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# 학습된 모델 저장
model_path = '/content/drive/MyDrive/2024 덕성여대 졸업 프로젝트/model.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved to {model_path}')

# 나중에 모델을 로드하여 사용
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

# 이미지 예측 함수 정의
def predict_image(image_path, model, transform, classes):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)  # 배치 차원 추가

    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output.data, 1)

    return predicted.item()

# 클래스 라벨 정의
classes = ['notEnter', 'notLeft', 'right', 'slow']

# 이미지 경로 설정 (다운로드 폴더 내의 이미지 경로)
image_folder = '/content/drive/MyDrive/2024 덕성여대 졸업 프로젝트/표지판 데이터셋/CNN 데이터셋/testCNN_Nomal'  # 실제 이미지 폴더 경로
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('png', 'jpg', 'jpeg'))]

# 각 이미지에 대해 예측 수행 및 출력
for image_path in image_paths:
    predicted_class = predict_image(image_path, model, valid_test_transform, classes)
    print(f'Image: {os.path.basename(image_path)} -> Predicted Class: {predicted_class}')

# 예측된 클래스와 실제 클래스를 비교하여 출력
predicted_classes = []
true_classes = []
for inputs, labels in test_loader:
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    predicted_classes.extend(predicted.tolist())  # 예측된 클래스를 리스트에 추가
    true_classes.extend(labels.tolist())          # 실제 클래스를 리스트에 추가

for i, (predicted_class, true_class) in enumerate(zip(predicted_classes, true_classes)):
    predicted_sign = classes[predicted_class]
    true_sign = classes[true_class]
    print(f"테스트 이미지 {i+1}: 예측된 표지판 - {predicted_sign}, 실제 표지판 - {true_sign}")



Epoch [1/100], Loss: 0.7551
Validation Accuracy: 52.45%
Epoch [2/100], Loss: 0.1202
Validation Accuracy: 95.10%
Epoch [3/100], Loss: 0.1238
Validation Accuracy: 90.21%
Epoch [4/100], Loss: 0.0430
Validation Accuracy: 95.80%
Epoch [5/100], Loss: 0.0312
Validation Accuracy: 94.41%
Epoch [6/100], Loss: 0.0329
Validation Accuracy: 95.80%
Epoch [7/100], Loss: 0.1113
Validation Accuracy: 90.91%
Epoch [8/100], Loss: 0.0449
Validation Accuracy: 97.90%
Epoch [9/100], Loss: 0.0238
Validation Accuracy: 91.61%
Epoch [10/100], Loss: 0.0121
Validation Accuracy: 95.80%
Epoch [11/100], Loss: 0.0035
Validation Accuracy: 95.80%
Epoch [12/100], Loss: 0.0104
Validation Accuracy: 95.80%
Epoch [13/100], Loss: 0.0417
Validation Accuracy: 96.50%
Epoch [14/100], Loss: 0.0318
Validation Accuracy: 97.20%
Epoch [15/100], Loss: 0.0138
Validation Accuracy: 96.50%
Epoch [16/100], Loss: 0.0073
Validation Accuracy: 94.41%
Epoch [17/100], Loss: 0.0025
Validation Accuracy: 95.80%
Epoch [18/100], Loss: 0.0086
Validation 

<ipython-input-7-e41768abc5db>:103: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


테스트 이미지 1: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 2: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 3: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 4: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 5: 예측된 표지판 - notLeft, 실제 표지판 - notEnter
테스트 이미지 6: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 7: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 8: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 9: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 10: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 11: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 12: 예측된 표지판 - notEnter, 실제 표지판 - notEnter
테스트 이미지 13: 예측된 표지판 - notLeft, 실제 표지판 - notLeft
테스트 이미지 14: 예측된 표지판 - notLeft, 실제 표지판 - notLeft
테스트 이미지 15: 예측된 표지판 - notLeft, 실제 표지판 - notLeft
테스트 이미지 16: 예측된 표지판 - notLeft, 실제 표지판 - notLeft
테스트 이미지 17: 예측된 표지판 - notLeft, 실제 표지판 - notLeft
테스트 이미지 18: 예측된 표지판 - notLeft, 실제 표지판 - notLeft
테스트 이미지 19: 예측된 표지판 - notLeft, 실제 표지판 - notLeft
테스트 이미지 20: 예측된 표지판 - notLeft, 실제 표지판 - notLeft
테스트 이미지 21: 예측된 표지판 - notL